In [41]:
import google.generativeai as genai


In [42]:
# Guardar a API Key do Google em uma variável. IMPORTANTE: necessária para o funcionamento do chatbot. Obter em Google AI Studio (https://aistudio.google.com/app/apikey)
api_key_usuario = 'COLE_AQUI_A_SUA_API_KEY'

# Configuração da APi Key para o projeto
genai.configure(api_key=api_key_usuario)

In [43]:
# Configurações de Resposta
generation_config = {

      # A IA pode dar mais de uma resposta, ao colocar 1, haverá apenas uma opção de resposta
      'candidate_count':1,

      # Controla a criatividade. Quanto mais próximo de 1, mais criativo, porém menos previsível. Quanto mais próximo de 0, menos criativo, porém mais previsível
      'temperature':0.6,

      # Controla o número de palavras consideradas para completar a frase. Quanto maior o número, mais palavras seão consideradas. Valores maiores são mais criativos, porém menos precisos
      'top_k':30,

      # Controla a probabilidade total das palavras a serem consideradas para completar a frase. Quanto mais próximo de 1, maior a probabilidade e criatividade, enquanto o mais próximo de 0, menor será essas caractrísticas
      'top_p':0.8,

      # Define o número máximo de tokens gerados por resposta. Serão geradas em torno de 700 palavras
      'max_output_tokens':1024
}

In [44]:
# Configurações de Segurança
safety_settings = {

     # Bloqueio de Abuso
     'HARASSMENT': 'HIGH',

     # Bloqueio de Ódio
     'HATE': 'HIGH',

     # Bloqueio de conteúdo Sexual Explícito
     'SEXUAL': 'MEDIUM',

     # Bloqueio de Comentários Perigosos
     'DANGEROUS': 'MEDIUM'
}

In [ ]:
# Lista de modelos Gemini
for x in genai.list_models():
   if 'generateContent' in x.supported_generation_methods:
      print(x.name)

# Escolha e Inicialização do modelo (o modelo 1.5 flash será usado)
modelo_gemini = genai.GenerativeModel(model_name='gemini-1.5-flash', generation_config=generation_config, safety_settings=safety_settings)

In [46]:
# Interação Usuário-Chatbot

# Funções:
# Função para obter a entrada do usuário (pergunta)
def obter_entrada_usuario():
  return input(f'\n Digite a sua pergunta (ou s para sair): \n\n')

# Função para enviar a pergunta ao modelo
def enviar_pergunta(envio, entrada):
  return envio.send_message(entrada)

# Função para tratar exceções
def tratar_erro(excecao):
  return excecao

In [ ]:
# Interação Usuário-Chatbot

# Sessão de conversa e criação do histórico da conversa
conversa = modelo_gemini.start_chat(history=[])

# Loop para que a conversa se alongue enquanto o usuário desejar
while True:
  try:
    entrada_usuario = obter_entrada_usuario()

    if entrada_usuario.lower() == 's':
      print(f'\n Obrigado por usar o chatbot!')
      break

    # Envio da pergunta feita pelo usuário para o modelo

    resposta_chatbot = enviar_pergunta(conversa, entrada_usuario)

    # Exibição da resposta
    print(f'\n Resposta: {resposta_chatbot.text} \n\n')

  except Exception as e:
      # Conversão da exceção para uma string, para que possa ser procurado um texto específico (no caso, algo que viole as políticas de segurança)
      erro_str = str(tratar_erro(e))

      # 'finish_reason: SAFETY' simboliza que políticas de segurança foram violadas. Dessa forma, se houver um erro, dentro da exceção, que aponte isso, será enviada a notificação própria para o contexto

      if 'finish_reason: SAFETY' in erro_str:
          mensagem_erro_seguranca = '\n Sua pergunta viola políticas de segurança. Por favor, evite linguagem discriminatória, prejudicial ou ofensiva, e também conteúdos sensíveis como violência explícita e conteúdo sexual. \n\n'
          print(mensagem_erro_seguranca)

      # Se houver outros tipos de erros, será exibida a seguinte notificação:
      else:
          mensagem_erro_generico = f'\n {pular_linha} Desculpe, mas houve um erro. Por favor, faça sua pergunta novamente. {e} \n\n'
          print(mensagem_erro_generico)